## Notes

Questions:
    * Are we reading the raw lead correctly?
    * Do certain field values correlate with higher conversion?
        * Within Partner i.
        * Across Partners.

Additional Data:
    * URL
    * Click ID
    * Funnel Performance. Denial Reason, conversion boolean



LenderEdge
    * A. Raw DOB is standard date format, without dashes. A couple of blanks.
    * B. When raw_incometype != 'Employment', defaults to 'OtherTaxableIncome'. ~30k, check time series.
    * C. Often writes NA accountnumber when raw_accountnumber contains a value. ~500k, check time series.
    * D. Blank, != 9 digit, and mail merge fields passed into Routing Number.

1B) 
df.lenderedge.comparison %>%
    group_by(
        raw_incometype,
        incometype
    ) %>% 
    summarize(
        n()
    )

# Import Libraries

In [3]:
library(opploansanalytics)
load.packages()

RPostgreSQL    RPostgres        RJDBC         httr     jsonlite googlesheets 
        TRUE         TRUE         TRUE         TRUE         TRUE         TRUE 
   tidyverse        tidyr        dplyr    lubridate      stringr      ggplot2 
        TRUE         TRUE         TRUE         TRUE         TRUE         TRUE 
      ggpubr      cowplot        rJava        mailR     devtools     roxygen2 
        TRUE         TRUE         TRUE         TRUE         TRUE         TRUE

# Import and Clean Data

### Leads

In [3]:
getAdmethod = function () {
    
    queryReporting(
"
select
    c_adm.name
    , count(*)
from
    cloudlending.applications as c_app
    inner join
        cloudlending.advertising_method as c_adm
        on c_app.advertising_method = c_adm.id
        and right(c_adm.name,1) = '4'
where
    date_trunc('month', c_app.createddate) = '2019-09-01'::date
group by
    1
order by
    2 desc
limit 10    
"
    )
}

In [4]:
getLeadsDF = function (admethod.name, timestart, timeend) {
    
    queryReporting(
        paste0(
"
select

    --  Identifiers  --
    lde.lead_id
    , lde.partnerid
    , lde.leadofferid
    , lde.lead_time
    , c_am.name
    , lde.accepted
    , lde.reason
    , lde.offer_amount
    --  Banking  --
    , lde.abaroutingnumber
    , lde.accountnumber
    , lde.bankname
    --  Employment  --
    , lde.grossmonthlyincome
    , lde.incometype
    , lde.lastpayrolldate
    , lde.payrollfrequency
    , lde.payrolltype
    , lde.work_hiredate
    --  Contact  --
    , lde.dateofbirth
    , lde.statecode


    --  Raw Lead  --
    , lde.raw_lead
        --  ID  --
    , lde.raw_lead ->> 'leadOfferId' as raw_leadofferid
    , lde.raw_lead ->> 'requestedLoanAmount' as raw_loan_request
        --  Banking  --
    , lde.raw_lead -> 'bankInfo' ->> 'abaRoutingNumber' as raw_abaroutingnumber
    , lde.raw_lead -> 'bankInfo' ->> 'accountNumber' as raw_accountnumber
    , lde.raw_lead -> 'bankInfo' ->> 'bankName' as raw_bankname
    , lde.raw_lead -> 'bankInfo' ->> 'accountType' as raw_accounttype
    , lde.raw_lead -> 'bankInfo' ->> 'accountLength' as raw_accountlength
        --  Employment  --
    , lde.raw_lead ->> 'grossMonthlyIncome' as raw_grossmonthlyincome
    , lde.raw_lead -> 'incomeInfo' ->> 'incomeType' as raw_incometype
    , lde.raw_lead -> 'incomeInfo' ->> 'lastPayrollDate' as raw_lastpayrolldate
    , lde.raw_lead -> 'incomeInfo' ->> 'payrollFrequency' as raw_payrollfrequency
    , lde.raw_lead -> 'incomeInfo' ->> 'payrollType' as raw_payrolltype
    , lde.raw_lead -> 'employmentInfo' ->> 'hireDate' as raw_work_hiredate
        --  Contact  --
    , lde.raw_lead -> 'personalInfo' ->> 'dateOfBirth' as raw_dateofbirth
    , lde.raw_lead ->> 'stateCode' as raw_statecode
    from
        lde4.leads as lde
    inner join
        cloudlending.advertising_method as c_am
        on lde.partnerid = c_am.external_id
        and c_am.name = '", admethod.name, " 
    where
        lde.lead_time >= '", timestart, "'::date
        and lde.lead_time <= '", timeend, "'::date
"
        )
    )
}

In [5]:
formatDataTypes = function (df) {
    
    df.reformat = df %>%
        mutate_if(
            is.factor,
            as.character
        ) %>% 
        mutate(
            
#             dateofbirth = if_else(
#                 !is.na(dateofbirth) & dateofbirth != '',
#                 dateofbirth %>% as.POSIXct() %>% ceiling_date(unit = 'day') %>% as.character(),
#                 dateofbirth
#             ),
#             raw_dateofbirth = if_else(
#                 raw_dateofbirth != '',
#                 paste(
#                     raw_dateofbirth %>% str_sub(1,4),
#                     raw_dateofbirth %>% str_sub(5,6),
#                     raw_dateofbirth %>% str_sub(7,8),
#                     sep = '-'
#                 ),
#                 raw_dateofbirth
#             ),
            
            zip = if_else(
                zip %>% nchar() == 4,
                paste0('0', zip),
                zip %>% as.character()
            ),
            raw_zip = if_else(
                raw_zip %>% nchar() == 4,
                paste0('0', raw_zip),
                raw_zip %>% as.character()
            ),
            
            abaroutingnumber = abaroutingnumber %>% 
                str_pad(
                    width = 9,
                    pad = '0'
                ),
            raw_abaroutingnumber = raw_abaroutingnumber %>% 
                str_pad(
                    width = 9,
                    pad = '0'
                ),
            
            abaroutingnumber.is.valid = 
                grepl("^[0-9]{9}", abaroutingnumber) &
                !grepl("^[0]{8}[0-9]", abaroutingnumber) &
                !grepl("^(0)\\1\\1\\1\\1\\1\\1\\1\\1", abaroutingnumber) &
                (
                    abaroutingnumber %>% str_sub(1,1) %>% as.integer() * 3 +
                    abaroutingnumber %>% str_sub(2,2) %>% as.integer() * 7 +
                    abaroutingnumber %>% str_sub(3,3) %>% as.integer() * 1 +
                    abaroutingnumber %>% str_sub(4,4) %>% as.integer() * 3 +
                    abaroutingnumber %>% str_sub(5,5) %>% as.integer() * 7 +
                    abaroutingnumber %>% str_sub(6,6) %>% as.integer() * 1 +
                    abaroutingnumber %>% str_sub(7,7) %>% as.integer() * 3 +
                    abaroutingnumber %>% str_sub(8,8) %>% as.integer() * 7 +
                    abaroutingnumber %>% str_sub(9,9) %>% as.integer() * 1
                ) %% 10 == 0,
            raw_abaroutingnumber.is.valid = 
                grepl("^[0-9]{9}", raw_abaroutingnumber) &
                !grepl("^[0]{8}[0-9]", raw_abaroutingnumber) &
                !grepl("^(0)\\1\\1\\1\\1\\1\\1\\1\\1", raw_abaroutingnumber) &
                (
                    raw_abaroutingnumber %>% str_sub(1,1) %>% as.integer() * 3 +
                    raw_abaroutingnumber %>% str_sub(2,2) %>% as.integer() * 7 +
                    raw_abaroutingnumber %>% str_sub(3,3) %>% as.integer() * 1 +
                    raw_abaroutingnumber %>% str_sub(4,4) %>% as.integer() * 3 +
                    raw_abaroutingnumber %>% str_sub(5,5) %>% as.integer() * 7 +
                    raw_abaroutingnumber %>% str_sub(6,6) %>% as.integer() * 1 +
                    raw_abaroutingnumber %>% str_sub(7,7) %>% as.integer() * 3 +
                    raw_abaroutingnumber %>% str_sub(8,8) %>% as.integer() * 7 +
                    raw_abaroutingnumber %>% str_sub(9,9) %>% as.integer() * 1
                ) %% 10 == 0,
            
            accountnumber.is.valid = 
                grepl("^[0-9]{6,17}", accountnumber) &
                !grepl("^([0-9])\\1\\1\\1\\1\\1\\1?\\1?\\1?\\1?\\1?\\1?\\1?\\1?\\1?\\1?\\1?", accountnumber),
            raw_accountnumber.is.valid = 
                grepl("^[0-9]{6,17}", raw_accountnumber) &
                !grepl("^([0-9])\\1\\1\\1\\1\\1\\1?\\1?\\1?\\1?\\1?\\1?\\1?\\1?\\1?\\1?\\1?", raw_accountnumber)
            
        )
    
#     df.reformat.no.na = df.reformat %>%
#         apply(
#             MARGIN = 2,
#             replace_na,
#             replace = ''
#         )
    
    return(df.reformat)
}

In [6]:
compareExtractionToRaw = function (df) {
    
    df %>% 
        mutate(
            match.leadofferid = 
                (is.na(leadofferid) & is.na(raw_leadofferid)) |
                (is.na(leadofferid) & raw_leadofferid == '') |
                (!is.na(leadofferid) & !is.na(raw_leadofferid) & leadofferid == raw_leadofferid),
            
            match.abaroutingnumber = 
                (is.na(abaroutingnumber) & is.na(raw_abaroutingnumber)) |
                (is.na(abaroutingnumber) & raw_abaroutingnumber == '') |
                (!is.na(abaroutingnumber) & !is.na(raw_abaroutingnumber) & abaroutingnumber == raw_abaroutingnumber),
            
            match.accountnumber = 
                (is.na(accountnumber) & is.na(raw_accountnumber)) |
                (is.na(accountnumber) & raw_accountnumber == '') |
                (!is.na(accountnumber) & !is.na(raw_accountnumber) & accountnumber == raw_accountnumber),
            
            match.bankname = 
                (is.na(bankname) & is.na(raw_bankname)) |
                (is.na(bankname) & raw_bankname == '') |
                (!is.na(bankname) & !is.na(raw_bankname) & bankname == raw_bankname),
            
            match.grossmonthlyincome = 
                (is.na(grossmonthlyincome) & is.na(raw_grossmonthlyincome)) |
                (is.na(grossmonthlyincome) & raw_grossmonthlyincome == '') |
                (!is.na(grossmonthlyincome) & !is.na(raw_grossmonthlyincome) & grossmonthlyincome == raw_grossmonthlyincome),
            
            match.incometype = 
                (is.na(incometype) & is.na(raw_incometype)) |
                (is.na(incometype) & raw_incometype == '') |
                (!is.na(incometype) & !is.na(raw_incometype) & incometype == raw_incometype),
            
            match.lastpayrolldate = 
                (is.na(lastpayrolldate) & is.na(raw_lastpayrolldate)) |
                (is.na(lastpayrolldate) & raw_lastpayrolldate == '') |
                (!is.na(lastpayrolldate) & !is.na(raw_lastpayrolldate) & lastpayrolldate == raw_lastpayrolldate),
            
            match.payrollfrequency = 
                (is.na(payrollfrequency) & is.na(raw_payrollfrequency)) |
                (is.na(payrollfrequency) & raw_payrollfrequency == '') |
                (!is.na(payrollfrequency) & !is.na(raw_payrollfrequency) & payrollfrequency == raw_payrollfrequency),
            
            match.payrolltype = 
                (is.na(payrolltype) & is.na(raw_payrolltype)) |
                (is.na(payrolltype) & raw_payrolltype == '') |
                (!is.na(payrolltype) & !is.na(raw_payrolltype) & payrolltype == raw_payrolltype),
            
            match.work_hiredate = 
                (is.na(work_hiredate) & is.na(raw_work_hiredate)) |
                (is.na(work_hiredate) & raw_work_hiredate == '') |
                (!is.na(work_hiredate) & !is.na(raw_work_hiredate) & work_hiredate == raw_work_hiredate),
            
            match.dateofbirth = 
                (is.na(dateofbirth) & is.na(raw_dateofbirth)) |
                (is.na(dateofbirth) & raw_dateofbirth == '') |
                (!is.na(dateofbirth) & !is.na(raw_dateofbirth) & dateofbirth == raw_dateofbirth),
            
            match.statecode = 
                (is.na(statecode) & is.na(raw_statecode)) |
                (is.na(statecode) & raw_statecode == '') |
                (!is.na(statecode) & !is.na(raw_statecode) & statecode == raw_statecode)
        )
    
}

In [7]:
examineMatches = function (df) {

    df %>%
        select(
            starts_with('match.')
        ) %>% 
        apply(
            MARGIN = 2,
            FUN = sum
        ) %>% sort() %>%
        c(
            nrow = df.lenderedge.comparison %>% nrow()
        )
}

### Funnel

In [ ]:
getResponseSubset = function (df) {
    
    df %>%
#         filter(
#             appldate >= '2019-07-01' %>% as.Date() &
#             appldate < '2019-10-01' %>% as.Date()
#         ) %>%
        mutate(
            click_id_pos = partner_event_data %>%
                str_extract(
                    "click_id=%?[0-9]+"
                ) %>% 
                str_locate(
                    "="
                ) %>% .[, 1],
            click_id = partner_event_data %>%
                str_extract(
                    "click_id=%?[0-9]+"
                ) %>% 
                str_sub(
                    start = click_id_pos + 1,
                    end = partner_event_data %>% nchar()
                ) %>%
                replace_na(
                    'None'
                ),
            has.unique.lead.id = !(lead_id %in% (
                df %>%
                    select(lead_id) %>%
                    group_by(lead_id) %>%
                    mutate(n = n()) %>%
                    ungroup() %>% 
                    filter(n > 1) %>% 
                    .$lead_id
            ))
        )
    
}

### Map

In [9]:
library(zipcode)
library(tidyverse)
library(maps)
library(viridis)
library(ggthemes)
library(albersusa)
# https://austinwehrwein.com/digital-humanities/creating-a-density-map-in-r-with-zipcodes/

In [148]:
us = ggplot2::map_data('state')
us2 = ggplot2::map_data('county')

In [135]:
zip.codes = read_csv("../data/zip_code_database.csv")

Parsed with column specification:
cols(
  zip = col_character(),
  type = col_character(),
  decommissioned = col_double(),
  primary_city = col_character(),
  acceptable_cities = col_character(),
  unacceptable_cities = col_character(),
  state = col_character(),
  county = col_character(),
  timezone = col_character(),
  area_codes = col_number(),
  world_region = col_logical(),
  country = col_character(),
  latitude = col_double(),
  longitude = col_double(),
  irs_estimated_population_2015 = col_double()
)
Warning message:
"333 parsing failures.
 row          col           expected actual                            file
3242 world_region 1/0/T/F/TRUE/FALSE     EU '../data/zip_code_database.csv'
3245 world_region 1/0/T/F/TRUE/FALSE     EU '../data/zip_code_database.csv'
3246 world_region 1/0/T/F/TRUE/FALSE     EU '../data/zip_code_database.csv'
3247 world_region 1/0/T/F/TRUE/FALSE     EU '../data/zip_code_database.csv'
3248 world_region 1/0/T/F/TRUE/FALSE     EU '../data/zip_code_d

In [136]:
valid.zip = zip.codes %>%
    filter(
        !state %in% c('AA', 'AE', 'AP', 'AS', 'FM', 'GU', 'MH', 'MP', 'PR', 'PW', 'VI') &
        decommissioned == 0 &
        nchar(zip) == 5
    ) %>% 
    select(
        zip,
        state,
        latitude,
        longitude
    )

# LenderEdge 4

Proposal:
- Income:
    - <b>Ignore the value passed in the payload.</b>
    - Higher income customer to reduce Insiffucient Income denials.
- DOB
    - Senior applicants have more issues through the funnel
- <b>Click ID!</b>
- <b>Wait for more data</b>, to create a more robust decision tree/random forest model.
    - Initial model suggests to accept the following income ranges & click IDs for each state:

In [ ]:
# df.lenderedge.comparison = read.csv(
#         "..\\data\\lenderedge4_df.csv"
#     ) %>%
#     formatDataTypes() %>%
#     compareExtractionToRaw()

# # df.lenderedge.comparison %>% examineMatches()

# lenderedge.response.subset = read_csv(
#         "../data/lenderedge4_response_df.csv"
#     ) %>% getResponseSubset()

# lenderedge = lenderedge.response.subset %>%
#     left_join(
#         df.lenderedge.comparison,
#         by = c('lead_id' = 'lead_id')
#     ) %>%
#     left_join(
#         valid.zip,
#         by = c('zip' = 'zip')
#     )

# lenderedge %>% write.csv("../data/lenderedge4_df.csv")

In [145]:
# lenderedge = read_csv("..\\data\\lenderedge4_df.csv")

In [6]:
theme_set(theme_bw())

### Time Series

- In October, Funding Rate increased, driven by BV/Q and F/Decisioned.
- In October, We began to ignore the Bank Account JSON Object in the raw_lead payload.
- Mean GMI has steadily increased, reflecting offer amount increase and Insufficient Income decrease.- 

In [56]:
getTimeSeriesFunnel = function (df, aspect.ratio) {
    
    df %>% 
        group_by(
            appldate_ = appldate %>% as.Date()
        ) %>% 
        summarize(
            app.to.funded = mean(funded),
            funding.rate = sum(funded)/sum(qualified),
            bv.q = sum(bankverified)/sum(qualified),
            sc.bv = sum(passscorecardratecard)/sum(bankverified),
            cs.sc = sum(contractsigned)/sum(passscorecardratecard),
            dec.cs = sum(cs_decisioned)/sum(contractsigned),
            f.dec = sum(funded)/sum(cs_decisioned)
        ) %>% 
        ggplot(
            mapping = aes(
                x = appldate_
            )
        ) +
        geom_point(aes(y = app.to.funded), color = 'gray') +
        geom_line(aes(y = app.to.funded), color = 'gray') +
        geom_smooth(aes(y = app.to.funded), color = 'gray') +

        geom_point(aes(y = funding.rate), color = 'black') +
        geom_line(aes(y = funding.rate), color = 'black') +
        geom_smooth(aes(y = funding.rate), color = 'black') +

        geom_point(aes(y = bv.q), color = 'red') +
        geom_line(aes(y = bv.q), color = 'red') +
        geom_smooth(aes(y = bv.q), color = 'red') +

        geom_point(aes(y = sc.bv), color = 'blue') +
        geom_line(aes(y = sc.bv), color = 'blue') +
        geom_smooth(aes(y = sc.bv), color = 'blue') +

        geom_point(aes(y = cs.sc), color = 'darkgreen') +
        geom_line(aes(y = cs.sc), color = 'darkgreen') +
        geom_smooth(aes(y = cs.sc), color = 'darkgreen') +

        geom_point(aes(y = dec.cs), color = 'darkorange') +
        geom_line(aes(y = dec.cs), color = 'darkorange') +
        geom_smooth(aes(y = dec.cs), color = 'darkorange') +

        geom_point(aes(y = f.dec), color = 'purple') +
        geom_line(aes(y = f.dec), color = 'purple') +
        geom_smooth(aes(y = f.dec), color = 'purple') +


        scale_y_continuous(
            labels = scales::percent
        ) +
        labs(
            title = 'LenderEdge 4 Funding Time Series',
            subtitle = "Black: From Qualified\nGray: From App",
            y = 'Conversion',
            x = 'Appldate'
        ) +
        theme(
            plot.title = element_text(hjust = 0.5),
            plot.subtitle = element_text(hjust = 0.5),
        ) +
        coord_fixed(
            ratio = aspect.ratio
        )
    
}
# lenderedge %>% getTimeSeriesFunnel(90)

In [87]:
getTimeSeriesGMI = function (df, aspect.ratio) {
    
    df %>% 
        filter(
            !is.na(grossmonthlyincome) &
            grossmonthlyincome < 50000
        ) %>% 
        group_by(
            appldate_ = appldate %>% as.Date()
        ) %>% 
        summarize(
            gmi.median = median(grossmonthlyincome),
            gmi.mean = mean(grossmonthlyincome),
            gmi.max = max(grossmonthlyincome)
        ) %>% 
        ggplot(
            mapping = aes(
                x = appldate_
            )
        ) +

        geom_point(aes(y = gmi.mean), color = 'black') +
        geom_line(aes(y = gmi.mean), color = 'black') +
        geom_smooth(aes(y = gmi.mean), color = 'black') +

        geom_point(aes(y = gmi.median), color = 'red') +
        geom_line(aes(y = gmi.median), color = 'red') +
        geom_smooth(aes(y = gmi.median), color = 'red') +

        scale_y_continuous(
            labels = scales::dollar
        ) +
        labs(
            title = 'LenderEdge 4 Income Time Series',
            subtitle = "Black: Mean\nRed: Median",
            y = 'Gross Monthly Income',
            x = 'Appldate'
        ) +
        theme(
            plot.title = element_text(hjust = 0.5),
            plot.subtitle = element_text(hjust = 0.5),
        ) +
        coord_fixed(
            ratio = aspect.ratio
        )
}
# lenderedge %>% getTimeSeriesGMI(0.03)

In [99]:
getTimeSeriesOfferAmount = function (df, aspect.ratio) {
    
    df %>% 
        filter(
            !is.na(offer_amount)
        ) %>% 
        group_by(
            appldate_ = appldate %>% as.Date()
        ) %>% 
        summarize(
            offer.median = median(offer_amount),
            offer.mean = mean(offer_amount)
        ) %>% 
        ggplot(
            mapping = aes(
                x = appldate_
            )
        ) +

        geom_point(aes(y = offer.mean), color = 'black') +
        geom_line(aes(y = offer.mean), color = 'black') +
        geom_smooth(aes(y = offer.mean), color = 'black') +

        geom_point(aes(y = offer.median), color = 'red') +
        geom_line(aes(y = offer.median), color = 'red') +
        geom_smooth(aes(y = offer.median), color = 'red') +

        scale_y_continuous(
            labels = scales::dollar
        ) +
        labs(
            title = 'LenderEdge 4 Offer Amount Time Series',
            subtitle = "Black: Mean\nRed: Median",
            y = 'Gross Monthly Income',
            x = 'Appldate'
        ) +
        theme(
            plot.title = element_text(hjust = 0.5),
            plot.subtitle = element_text(hjust = 0.5),
        ) +
        coord_fixed(
            ratio = aspect.ratio
        )
}
# lenderedge %>% getTimeSeriesOfferAmount(0.1)

In [142]:
getTimeSeriesValidBank = function (df, aspect.ratio) {
    
    df %>% 
        filter(
            !is.na(raw_abaroutingnumber.is.valid) &
            !is.na(raw_accountnumber.is.valid)
        ) %>% 
        group_by(
            appldate_ = appldate %>% as.Date()
        ) %>% 
        summarize(
            routing = mean(raw_abaroutingnumber.is.valid),
            account = mean(raw_accountnumber.is.valid)
        ) %>% 
        ggplot(
            mapping = aes(
                x = appldate_
            )
        ) +

        geom_point(aes(y = routing), color = 'red') +
        geom_line(aes(y = routing), color = 'red') +
        geom_smooth(aes(y = routing), color = 'red') +

        geom_point(aes(y = account), color = 'black') +
        geom_line(aes(y = account), color = 'black') +
        geom_smooth(aes(y = account), color = 'black') +

        scale_y_continuous(
            labels = scales::percent
        ) +
        labs(
            title = 'LenderEdge 4 Valid Bank Time Series',
            subtitle = "Black: Account\nRed: Routing",
            y = '%Valid',
            x = 'Appldate'
        ) +
        theme(
            plot.title = element_text(hjust = 0.5),
            plot.subtitle = element_text(hjust = 0.5),
        ) +
        coord_fixed(
            ratio = aspect.ratio
        )
}
# lenderedge %>% getTimeSeriesValidBank(15)

In [144]:
getTimeSeriesDenialReason = function (df, aspect.ratio) {
    
    df %>%
        filter(
            cs_decisioned == 1
        ) %>% 
        mutate(
            denial_reason_agg = case_when(
                is.na(denial_reason)  ~  'Loan Approved',
                denial_reason %>% str_detect(pattern = '.*Insufficient Income.*')  ~  'Insufficient Income',
                denial_reason %>% str_detect(pattern = '.*No Direct Deposit.*')  ~  'No Direct Deposit',
                denial_reason %>% str_detect(pattern = '.*Incompatible Bank.*')  ~  'Incompatible Bank',
                denial_reason %>% str_detect(pattern = '.*Negative Bank Account Balance.*')  ~  'Negative Bank Account Balance',
                denial_reason %>% str_detect(pattern = '.*Self-Employed.*')  ~  'Self Employed'
            ) %>% replace_na('Other')
        ) %>% 
        group_by(
            appldate_ = appldate %>% as.Date()
        ) %>% 
        summarize(
            insufficient.income = mean(denial_reason_agg == 'Insufficient Income'),
            no.direct.deposit = mean(denial_reason_agg == 'No Direct Deposit'),
            incompatible.bank = mean(denial_reason_agg == 'Incompatible Bank'),
            negative.balance = mean(denial_reason_agg == 'Negative Bank Account Balance'),
            self.employed = mean(denial_reason_agg == 'Self Employed'),
        ) %>% 
        ggplot(
            mapping = aes(
                x = appldate_
            )
        ) +

        geom_point(aes(y = insufficient.income), color = 'red') +
        geom_line(aes(y = insufficient.income), color = 'red') +
        geom_smooth(aes(y = insufficient.income), color = 'red') +

        geom_point(aes(y = no.direct.deposit), color = 'black') +
        geom_line(aes(y = no.direct.deposit), color = 'black') +
        geom_smooth(aes(y = no.direct.deposit), color = 'black') +

#         geom_point(aes(y = incompatible.bank), color = 'blue') +
#         geom_line(aes(y = incompatible.bank), color = 'blue') +
#         geom_smooth(aes(y = incompatible.bank), color = 'blue') +

#         geom_point(aes(y = negative.balance), color = 'darkgreen') +
#         geom_line(aes(y = negative.balance), color = 'darkgreen') +
#         geom_smooth(aes(y = negative.balance), color = 'darkgreen') +

#         geom_point(aes(y = self.employed), color = 'purple') +
#         geom_line(aes(y = self.employed), color = 'purple') +
#         geom_smooth(aes(y = self.employed), color = 'purple') +

        scale_y_continuous(
            labels = scales::percent
        ) +
        labs(
            title = 'LenderEdge 4 Denial Reason Time Series',
            subtitle = "Black: No Direct Deposit\nRed: Insufficient Income",
            y = '%CS Decisioned',
            x = 'Appldate'
        ) +
        theme(
            plot.title = element_text(hjust = 0.5),
            plot.subtitle = element_text(hjust = 0.5),
        ) +
        coord_fixed(
            ratio = aspect.ratio
        )
}
# lenderedge %>% getTimeSeriesDenialReason(150)

### Geography

- Volume centralized in FL, Midwest, and West coast, each of which have a gradient for FR.
- There is no concentrated state that funds better than others.
- Most states/MSA show low conversion (red) on low volume (small).

In [154]:
getZipPlotVolume = function (df) {
    
    ##  With full df, sum Qualified Volume by Zip Code.  ##
    df %>%
        group_by(
            zip,
            longitude,
            latitude
        ) %>% 
        summarize(
            metric = sum(qualified)
        ) %>% 
        ##  Plot geographic coodinates as cartesian points  ##
        ggplot(
            mapping = aes(
                x = longitude,
                y = latitude
            )
        ) +
        ##  Gray county lines and black state lines  ##
        geom_polygon(
            data = us2,
            mapping =
                aes(
                    x = long,
                    y = lat,
                    group = group
                ),
                color = 'gray',
                fill = NA,
                alpha = 0.35
        ) +
        geom_polygon(
            data = us,
            mapping =
                aes(
                    x = long,
                    y = lat,
                    group = group
                ),
                color = 'black',
                fill = NA,
                alpha = 0.35
        ) +
        ##  Place point for country with color gradient on the metric (Q Volume)  ##
        geom_point(
            mapping = aes(
                color = metric
            ),
            size = 0.8,
            alpha = 0.25
        ) +
        xlim(-125, -65) +
        ylim(25, 50) +
        labs(
            title = "LenderEdge 4: New Qualified Volume by Geography"
        ) +
        ##  Adjust sizing  ##
        coord_fixed(ratio = 16/9)
    
}
# lenderedge %>% getZipPlotVolume()

In [155]:
getZipPlotFunnel = function (df, numerator.quo, denominator.quo) {
    
    ##  With full df, sum Qualified Volume by Zip Code.  ##
    df %>%
        filter(
            qualified == 1
        ) %>% 
        group_by(
            zip,
            longitude,
            latitude
        ) %>% 
        summarize(
            n = n(),
            metric = sum(!!numerator.quo)/sum(!!denominator.quo)
        ) %>% 
        ungroup() %>% 
        filter(
            n >= 1 &
            metric < 0.33 
#             metric > 0
        ) %>% 
        ##  Plot geographic coodinates as cartesian points  ##
        ggplot(
            mapping = aes(
                x = longitude,
                y = latitude
            )
        ) +
        ##  Gray county lines and black state lines  ##
        geom_polygon(
            data = us2,
            mapping =
                aes(
                    x = long,
                    y = lat,
                    group = group
                ),
                color = 'gray',
                fill = NA,
                alpha = 0.35
        ) +
        geom_polygon(
            data = us,
            mapping =
                aes(
                    x = long,
                    y = lat,
                    group = group
                ),
                color = 'black',
                fill = NA,
                alpha = 0.35
        ) +
        ##  Place point for country with color, size, and transparency gradient on the metric (FR)  ##
        geom_point(
            mapping = aes(
                color = metric,
                size = n,
#                 alpha = metric
            ),
            # size = 0.8,
            alpha = 0.6
        ) +
        xlim(-125, -65) +
        ylim(25, 50) +
        labs(
            title = "LenderEdge 4: New FR by Geography"
        ) +
        ##  Adjust sizing  ##
        coord_fixed(ratio = 16/9) +
        ##  Adjust coloring  ##
        scale_colour_gradientn(colours = c("brown3", "gold", "springgreen4"))
}
# lenderedge %>% getZipPlotFunnel(quo(funded), quo(qualified))

In [163]:
getZipPlotIncome = function (df) {
    
    ##  With full df, sum Qualified Volume by Zip Code.  ##
    df %>%
        filter(
            qualified == 1
        ) %>% 
        group_by(
            zip,
            longitude,
            latitude
        ) %>% 
        summarize(
            n = n(),
            metric = median(grossmonthlyincome)
        ) %>% 
        ungroup() %>% 
        filter(
            n >= 1 &
            metric <= 6000
        ) %>% 
        ##  Plot geographic coodinates as cartesian points  ##
        ggplot(
            mapping = aes(
                x = longitude,
                y = latitude
            )
        ) +
        ##  Gray county lines and black state lines  ##
        geom_polygon(
            data = us2,
            mapping =
                aes(
                    x = long,
                    y = lat,
                    group = group
                ),
                color = 'gray',
                fill = NA,
                alpha = 0.35
        ) +
        geom_polygon(
            data = us,
            mapping =
                aes(
                    x = long,
                    y = lat,
                    group = group
                ),
                color = 'black',
                fill = NA,
                alpha = 0.35
        ) +
        ##  Place point for country with color, size, and transparency gradient on the metric (FR)  ##
        geom_point(
            mapping = aes(
                color = metric,
                size = n,
                alpha = metric
            )
            # size = 0.8,
            # alpha = 0.25
        ) +
        xlim(-125, -65) +
        ylim(25, 50) +
#         labs(
#             title = "LenderEdge 4: New FR by Geography"
#         ) +
        ##  Adjust sizing  ##
        coord_fixed(ratio = 16/9) +
        ##  Adjust coloring  ##
        scale_colour_gradientn(colours = c("brown3", "gold", "springgreen4"))
}
# lenderedge %>% getZipPlotIncome()

In [138]:
getValidZipPlotFunnel = function (df, numerator.quo, denominator.quo) {

    ##  With full df, sum Qualified Volume by State.  ##
    summary.df = df %>%
        filter(
            qualified == 1
        ) %>% 
        group_by(
            is.valid = zip %in% valid.zip$zip
        ) %>% 
        summarize(
            n = n(),
            metric = sum(!!numerator.quo)/sum(!!denominator.quo)
        ) %>% 
        ungroup() %>% 
        filter(
            n >= 1 &
            metric < 1
        )
    
    ##  Bar Plot  ##
    summary.df %>% 
        ggplot(
            mapping = aes(
                x = is.valid,
                y = metric
            )
        ) +
        geom_col(
            mapping = aes(
                fill = n
            )
        ) + 
        ##  Adjust sizing  ##
        coord_fixed(
            ratio = 5
        ) +
        ##  Adjust coloring  ##
        scale_fill_gradient(
            low = "lightgray",
            high = "steelblue"
        )
    
}
# lenderedge %>% getValidZipPlotFunnel(quo(funded), quo(qualified))

In [165]:
getStatePlotFunnel = function (df, numerator.quo, denominator.quo) {

    ##  With full df, sum Qualified Volume by State.  ##
    summary.df = df %>%
        filter(
            qualified == 1
        ) %>% 
        group_by(
            statecode
        ) %>% 
        summarize(
            n = n(),
            metric = sum(!!numerator.quo)/sum(!!denominator.quo)
        ) %>% 
        ungroup() %>% 
        filter(
            n >= 1 &
            metric < 1
        )

    ##  Assign as factor and sort levels to assign ordering for plot.  ##
    summary.df$statecode = factor(summary.df$statecode, levels = summary.df$statecode[order(-summary.df$n)])
    
    ##  Bar Plot  ##
    summary.df %>% 
        ggplot(
            mapping = aes(
                x = statecode,
                y = metric
            )
        ) +
        geom_col(
            mapping = aes(
                fill = n
            )
        ) + 
        ##  Adjust sizing  ##
        coord_fixed(
            ratio = 40
        ) +
        ##  Adjust coloring  ##
        scale_fill_gradient(
            low = "lightgray",
            high = "steelblue"
        )
    
}
# lenderedge %>% getStatePlotFunnel(quo(funded), quo(qualified))

In [207]:
getZipGMIPlotFunnel = function (df, numerator.quo, denominator.quo) {
    
    ##  With full df, sum Qualified Volume by Zip Code.  ##
    df %>%
        filter(
            qualified == 1
        ) %>% 
        group_by(
            zip,
            longitude,
            latitude
        ) %>% 
        summarize(
            n = n(),
            metric = sum(!!numerator.quo)/sum(!!denominator.quo),
            variable = median(grossmonthlyincome)
        ) %>% 
        ungroup() %>% 
        filter(
            n >= 1 &
            metric < 0.4 &
            metric > 0
        ) %>% 
        ##  Plot geographic coodinates as cartesian points  ##
        ggplot(
            mapping = aes(
                x = longitude,
                y = latitude
            )
        ) +
        ##  Gray county lines and black state lines  ##
        geom_polygon(
            data = us2,
            mapping =
                aes(
                    x = long,
                    y = lat,
                    group = group
                ),
                color = 'gray',
                fill = NA,
                alpha = 0.35
        ) +
        geom_polygon(
            data = us,
            mapping =
                aes(
                    x = long,
                    y = lat,
                    group = group
                ),
                color = 'black',
                fill = NA,
                alpha = 0.35
        ) +
        ##  Place point for country with color, size, and transparency gradient on the metric (FR)  ##
        geom_point(
            mapping = aes(
                color = metric,
                size = variable,
#                 alpha = metric
            ),
            # size = 0.8,
            alpha = 0.7
        ) +
        xlim(-125, -65) +
        ylim(25, 50) +
#         labs(
#             title = "LenderEdge 4: New FR by Geography"
#         ) +
        ##  Adjust sizing  ##
        coord_fixed(ratio = 16/9) +
        ##  Adjust coloring  ##
        scale_colour_gradientn(colours = c("brown3", "gold", "springgreen4"))
}
# lenderedge %>% getZipGMIPlotFunnel(quo(funded), quo(qualified))

In [208]:
getZipDOBPlotFunnel = function (df, numerator.quo, denominator.quo) {
    
    ##  With full df, sum Qualified Volume by Zip Code.  ##
    df %>%
        filter(
            qualified == 1 &
            !is.na(raw_dateofbirth)
        ) %>% 
        group_by(
            zip,
            longitude,
            latitude
        ) %>% 
        summarize(
            n = n(),
            metric = sum(!!numerator.quo)/sum(!!denominator.quo),
            variable = mean(((lead_time %>% as.Date() - raw_dateofbirth %>% as.Date()) %>% as.numeric())/365)
        ) %>% 
        ungroup() %>% 
        filter(
            n >= 1 &
            metric < 1 &
            metric > 0
        ) %>% 
        ##  Plot geographic coodinates as cartesian points  ##
        ggplot(
            mapping = aes(
                x = longitude,
                y = latitude
            )
        ) +
        ##  Gray county lines and black state lines  ##
        geom_polygon(
            data = us2,
            mapping =
                aes(
                    x = long,
                    y = lat,
                    group = group
                ),
                color = 'gray',
                fill = NA,
                alpha = 0.35
        ) +
        geom_polygon(
            data = us,
            mapping =
                aes(
                    x = long,
                    y = lat,
                    group = group
                ),
                color = 'black',
                fill = NA,
                alpha = 0.35
        ) +
        ##  Place point for country with color, size, and transparency gradient on the metric (FR)  ##
        geom_point(
            mapping = aes(
                color = metric,
                size = variable,
#                 alpha = metric
            ),
            # size = 0.8,
            alpha = 0.7
        ) +
        xlim(-125, -65) +
        ylim(25, 50) +
#         labs(
#             title = "LenderEdge 4: New FR by Geography"
#         ) +
        ##  Adjust sizing  ##
        coord_fixed(ratio = 16/9) +
        ##  Adjust coloring  ##
        scale_colour_gradientn(colours = c("brown3", "gold", "springgreen4"))
}
# lenderedge %>% getZipDOBPlotFunnel(quo(funded), quo(qualified))
# lenderedge %>% getZipDOBPlotFunnel(quo(bankverified), quo(qualified))

### Income

- Concentrated in Lower Income
- Passes in discrete values, can we ask for Income again?
- FR for higher income customers becomes more volatile.

In [172]:
getIncomePlotFunnel = function (df, numerator.quo, denominator.quo) {

#     ##  With full df, sum Qualified Volume by State.  ##
#     summary.df = df %>%
#         filter(
#             qualified == 1
#         ) %>% 
#         group_by(
#             grossmonthlyincome = .$grossmonthlyincome/100,
#             grossmonthlyincome = grossmonthlyincome %>% floor() * 100
#         ) %>% 
#         summarize(
#             n = n(),
#             metric = sum(!!numerator.quo)/sum(!!denominator.quo)
#         ) %>% 
#         ungroup() %>% 
#         filter(
#             n >= 1 &
#             metric < 1
#         )
    
#     ##  Bar Plot  ##
#     summary.df %>% 
#         ggplot(
#             mapping = aes(
#                 x = grossmonthlyincome,
#                 y = metric
#             )
#         ) +
#         geom_col(
#             mapping = aes(
#                 fill = n
#             )
#         ) + 
#         scale_y_continuous(
#             labels = scales::percent
#         ) +
#         xlim(0,8000) +
#         ##  Adjust sizing  ##
#         coord_fixed(
#             ratio = 5000
#         ) +
#         labs(
#             title = 'LenderEdge 4: New FR by GMI'
#         ) +
#         ##  Adjust coloring  ##
#         scale_fill_gradient(
#             low = "lightgray",
#             high = "steelblue"
#         )
    
}
# lenderedge %>% getIncomePlotFunnel(quo(funded), quo(qualified))
# # lenderedge.train %>% getIncomePlotFunnel(quo(funded), quo(qualified))
# # lenderedge.test %>% getIncomePlotFunnel(quo(funded), quo(qualified))

In [273]:
getGMIPlotDOB = function (df, numerator.quo, denominator.quo) {

    ##  With full df, sum Qualified Volume by State.  ##
    summary.df = df %>%
        filter(
            !is.na(dateofbirth) &
            !is.na(grossmonthlyincome) & 
            grossmonthlyincome < 50000
        ) %>% 
        group_by(
            dob = floor((appldate - dateofbirth) %>% as.integer()/365/10)*10
        ) %>% 
        summarize(
            n = n(),
            metric = sum(!!numerator.quo)/sum(!!denominator.quo),
            gmi = mean(grossmonthlyincome)
        ) %>% 
        ungroup() %>% 
        filter(
            n >= 1 &
            metric < 1
        )
    
    ##  Bar Plot  ##
    summary.df %>% 
        ggplot(
            mapping = aes(
                x = dob,
                y = gmi
            )
        ) +
        geom_col(
            mapping = aes(
                fill = n
            )
        ) + 
        ##  Adjust sizing  ##
        coord_fixed(
            ratio = 0.01
        ) +
        ##  Adjust coloring  ##
        scale_fill_gradient(
            low = "lightgray",
            high = "steelblue"
        ) +
        scale_y_continuous(
            labels = scales::dollar
        ) +
        geom_text(
            mapping = aes(
                label = n
            ),
            size = 3,
            angle = -45
        ) +
        theme(
            axis.text.x = element_text(
                angle = -45
            )
        )
    
}
# lenderedge %>% filter(appldate >= '2019-10-01' %>% as.Date()) %>% getGMIPlotDOB(quo(funded), quo(qualified))

In [266]:
getDOBGMITileFunnel = function (df, numerator.quo, denominator.quo) {

    ##  With full df, sum Qualified Volume by State.  ##
    summary.df = df %>%
        filter(
            !is.na(dateofbirth)
        ) %>% 
        group_by(
            dob = floor((appldate - dateofbirth) %>% as.integer()/365/10) * 10,
            gmi = floor(grossmonthlyincome/1000) * 1000
        ) %>% 
        summarize(
            n = n(),
            metric = sum(!!numerator.quo)/sum(!!denominator.quo)
        ) %>% 
        ungroup() %>% 
        filter(
            n >= 1 &
            metric < 0.3
        )
    
    ##  Bar Plot  ##
    summary.df %>% 
        ggplot(
            mapping = aes(
                x = gmi,
                y = dob
            )
        ) +
        geom_tile(
            mapping = aes(
                fill = metric
            )
        ) + 
        xlim(0,10000) +
        ##  Adjust sizing  ##
        coord_fixed(
            ratio = 80
        ) +
        ##  Adjust coloring  ##
        scale_fill_gradient(
            low = "lightgray",
            high = "steelblue"
        ) +
        geom_text(
            mapping = aes(
                label = n,
                size = n
            )
        )
    
}
# lenderedge %>% getDOBGMITileFunnel(quo(funded), quo(qualified))

In [268]:
getClickIDGMITileFunnel = function (df, numerator.quo, denominator.quo) {

    ##  With full df, sum Qualified Volume by State.  ##
    summary.df = df %>%
        filter(
            !is.na(dateofbirth)
        ) %>% 
        group_by(
            click_id,
            gmi = floor(grossmonthlyincome/1000) * 1000
        ) %>% 
        summarize(
            n = n(),
            metric = sum(!!numerator.quo)/sum(!!denominator.quo)
        ) %>% 
        ungroup() %>% 
        filter(
            n >= 1 &
            metric < 0.3
        )
    
    ##  Bar Plot  ##
    summary.df %>% 
        ggplot(
            mapping = aes(
                x = gmi,
                y = click_id
            )
        ) +
        geom_tile(
            mapping = aes(
                fill = metric
            )
        ) + 
        xlim(0,10000) +
        ##  Adjust sizing  ##
        coord_fixed(
            ratio = 350
        ) +
        ##  Adjust coloring  ##
        scale_fill_gradient(
            low = "lightgray",
            high = "steelblue"
        ) +
        geom_text(
            mapping = aes(
                label = n,
                size = n
            )
        )
    
}
# lenderedge %>% filter(appldate >= '2019-10-01' %>% as.Date()) %>%  getClickIDGMITileFunnel(quo(funded), quo(qualified))

In [ ]:
getClickIDGMITileDOB = function (df) {

    ##  With full df, sum Qualified Volume by State.  ##
    summary.df = df %>%
        filter(
            !is.na(dateofbirth) &
            !is.na(click_id) &
            !is.na(grossmonthlyincome)
        ) %>% 
        group_by(
            click_id,
            gmi = floor(grossmonthlyincome/1000) * 1000
        ) %>% 
        summarize(
            n = n(),
            dob = mean(floor((appldate - dateofbirth) %>% as.integer()/365) > 40)
        ) %>% 
        ungroup() %>% 
        filter(
            n >= 1
        )
    
    ##  Bar Plot  ##
    summary.df %>% 
        ggplot(
            mapping = aes(
                x = gmi,
                y = click_id
            )
        ) +
        geom_tile(
            mapping = aes(
                fill = dob
            )
        ) + 
        xlim(0,10000) +
        ##  Adjust sizing  ##
        coord_fixed(
            ratio = 350
        ) +
        ##  Adjust coloring  ##
        scale_fill_gradient(
            low = "lightgray",
            high = "steelblue"
        ) +
        geom_text(
            mapping = aes(
                label = n,
                size = n
            )
        )
    
}
# lenderedge %>% filter(appldate >= '2019-10-01' %>% as.Date()) %>%  getClickIDGMITileDOB()

### Click ID

- Click ID mix continuously shifts.
- How are Click IDs 1716 different from 1736 & 1757?
    - 1716 // 1736 on income
    - 1757 // 1716 & 1736 on income

In [247]:
getClickIDPlotFunnel = function (df, numerator.quo, denominator.quo) {

    ##  With full df, sum Qualified Volume by State.  ##
    summary.df = df %>%
        filter(
            qualified == 1 &
            !is.na(click_id)
        ) %>% 
        group_by(
            click_id
        ) %>% 
        summarize(
            n = n(),
            metric = sum(!!numerator.quo)/sum(!!denominator.quo)
        ) %>% 
        ungroup() %>% 
        filter(
            n >= 1 &
            metric < 1
        )
    
    summary.df$click_id = factor(summary.df$click_id, levels = summary.df$click_id[order(-summary.df$metric)])
    
    ##  Bar Plot  ##
    summary.df %>% filter(click_id != 'None') %>% 
        ggplot(
            mapping = aes(
                x = click_id,
                y = metric
            )
        ) +
        geom_col(
            mapping = aes(
                fill = n
            )
        ) + 
        ##  Adjust sizing  ##
        coord_fixed(
            ratio = 40
        ) +
        ##  Adjust coloring  ##
        scale_fill_gradient(
            low = "lightgray",
            high = "steelblue"
        ) +
        scale_y_continuous(
            labels = scales::percent
        ) +
        geom_text(
            mapping = aes(
                label = n
            ),
            size = 3,
            angle = -45
        ) +
        theme(
            axis.text.x = element_text(
                angle = -45
            )
        )
    
}
# lenderedge %>% filter(appldate >= '2019-10-01' %>% as.Date()) %>% getClickIDPlotFunnel(quo(funded), quo(qualified))

In [248]:
getClickIDPlotGMI = function (df, numerator.quo, denominator.quo) {

    ##  With full df, sum Qualified Volume by State.  ##
    summary.df = df %>%
        filter(
            !is.na(click_id) &
            !is.na(grossmonthlyincome)
        ) %>% 
        group_by(
            click_id
        ) %>% 
        summarize(
            n = n(),
            metric = sum(!!numerator.quo)/sum(!!denominator.quo),
            gmi = mean(grossmonthlyincome)
        ) %>% 
        ungroup() %>% 
        filter(
            n >= 1 &
            metric < 1
        )
    
    summary.df$click_id = factor(summary.df$click_id, levels = summary.df$click_id[order(-summary.df$metric)])
    
    ##  Bar Plot  ##
    summary.df %>% filter(click_id != 'None') %>% 
        ggplot(
            mapping = aes(
                x = click_id,
                y = gmi
            )
        ) +
        geom_col(
            mapping = aes(
                fill = n
            )
        ) + 
        ##  Adjust sizing  ##
        coord_fixed(
            ratio = 0.0015
        ) +
        ##  Adjust coloring  ##
        scale_fill_gradient(
            low = "lightgray",
            high = "steelblue"
        ) +
        scale_y_continuous(
            labels = scales::dollar
        ) +
        geom_text(
            mapping = aes(
                label = n
            ),
            size = 3,
            angle = -45
        ) +
        theme(
            axis.text.x = element_text(
                angle = -45
            )
        )
    
}
# lenderedge %>% filter(appldate >= '2019-10-01' %>% as.Date()) %>% getClickIDPlotGMI(quo(funded), quo(qualified))

In [250]:
getClickIDPlotDOB = function (df, numerator.quo, denominator.quo) {

    ##  With full df, sum Qualified Volume by State.  ##
    summary.df = df %>%
        filter(
            !is.na(click_id) &
            !is.na(dateofbirth)
        ) %>% 
        group_by(
            click_id
        ) %>% 
        summarize(
            n = n(),
            metric = sum(!!numerator.quo)/sum(!!denominator.quo),
#             dob = mean(floor((appldate - dateofbirth) %>% as.integer()/365))
            dob = mean(floor((appldate - dateofbirth) %>% as.integer()/365) <= 40)
        ) %>% 
        ungroup() %>% 
        filter(
            n >= 1 &
            metric < 1
        )
    
    summary.df$click_id = factor(summary.df$click_id, levels = summary.df$click_id[order(-summary.df$metric)])
    
    ##  Bar Plot  ##
    summary.df %>% filter(click_id != 'None') %>% 
        ggplot(
            mapping = aes(
                x = click_id,
                y = dob
            )
        ) +
        geom_col(
            mapping = aes(
                fill = n
            )
        ) + 
        ##  Adjust sizing  ##
        coord_fixed(
#             ratio = 0.12
            ratio = 8
        ) +
        ##  Adjust coloring  ##
        scale_fill_gradient(
            low = "lightgray",
            high = "steelblue"
        ) +
        scale_y_continuous(
#             labels = scales::dollar
        ) +
        geom_text(
            mapping = aes(
                label = n
            ),
            size = 3,
            angle = -45
        ) +
        theme(
            axis.text.x = element_text(
                angle = -45
            )
        )
    
}
# lenderedge %>% filter(appldate >= '2019-10-01' %>% as.Date()) %>% getClickIDPlotDOB(quo(funded), quo(qualified))

In [209]:
getClickIDTimeSeries = function (df, click.id.vector) {
    
    df %>% 
        filter(
            click_id %in% click.id.vector
        ) %>% 
        group_by(
            appldate_ = appldate %>% as.Date(),
            click_id
        ) %>% 
        summarize(
            n = n()
        ) %>% 
        ggplot(
            mapping = aes(
                x = appldate_,
                y = n
            )
        ) +
        geom_line(
            mapping = aes(
                color = click_id
            ),
            size = 1
        ) +
        geom_point(
            mapping = aes(
                color = click_id
            ),
            size = 2
        ) +
        coord_fixed(
            ratio = 0.1
        )
}
# lenderedge %>% getClickIDTimeSeries(click.id.vector = c('1716', '1736', '1757'))

### Bank

In [175]:
getABAPlotFunnel = function (df, numerator.quo, denominator.quo) {

    ##  With full df, sum Qualified Volume by State.  ##
    summary.df = df %>%
        mutate(
            na.aba = is.na(abaroutingnumber)
        ) %>% 
        filter(
            qualified == 1 &
            !is.na(abaroutingnumber.is.valid) &
            !(!abaroutingnumber.is.valid & !na.aba)
        ) %>% 
        group_by(
            enter.aba = abaroutingnumber.is.valid
        ) %>% 
        summarize(
            n = n(),
            metric = sum(!!numerator.quo)/sum(!!denominator.quo)
        ) %>% 
        ungroup() %>% 
        filter(
            n >= 1 &
            metric < 1
        )
    
    ##  Bar Plot  ##
    summary.df %>% 
        ggplot(
            mapping = aes(
                x = enter.aba,
                y = metric
            )
        ) +
        geom_col(
            mapping = aes(
                fill = n
            )
        ) + 
        ##  Adjust sizing  ##
        coord_fixed(
            ratio = 10
        ) +
        ##  Adjust coloring  ##
        scale_fill_gradient(
            low = "lightgray",
            high = "steelblue"
        )
    
}
# lenderedge %>% getABAPlotFunnel(quo(funded), quo(qualified))

In [176]:
getAccountNumPlotFunnel = function (df, numerator.quo, denominator.quo) {

    ##  With full df, sum Qualified Volume by State.  ##
    summary.df = df %>%
        filter(
            qualified == 1 &
            !is.na(accountnumber.is.valid) &
            !(!accountnumber.is.valid & is.na(accountnumber))
        ) %>% 
        group_by(
            accountnumber.is.valid
        ) %>% 
        summarize(
            n = n(),
            metric = sum(!!numerator.quo)/sum(!!denominator.quo)
        ) %>% 
        ungroup() %>% 
        filter(
            n >= 1 &
            metric < 1
        )
    
    ##  Bar Plot  ##
    summary.df %>% 
        ggplot(
            mapping = aes(
                x = accountnumber.is.valid,
                y = metric
            )
        ) +
        geom_col(
            mapping = aes(
                fill = n
            )
        ) + 
        ##  Adjust sizing  ##
        coord_fixed(
            ratio = 10
        ) +
        ##  Adjust coloring  ##
        scale_fill_gradient(
            low = "lightgray",
            high = "steelblue"
        )
    
}
# lenderedge %>% getAccountNumPlotFunnel(quo(funded), quo(qualified))

In [177]:
getAccountTypePlotFunnel = function (df, numerator.quo, denominator.quo) {

    ##  With full df, sum Qualified Volume by State.  ##
    summary.df = df %>%
        filter(
            qualified == 1
        ) %>% 
        group_by(
            raw_accounttype
        ) %>% 
        summarize(
            n = n(),
            metric = sum(!!numerator.quo)/sum(!!denominator.quo)
        ) %>% 
        ungroup() %>% 
        filter(
            n >= 1 &
            metric < 1
        )
    
    ##  Bar Plot  ##
    summary.df %>% 
        ggplot(
            mapping = aes(
                x = raw_accounttype,
                y = metric
            )
        ) +
        geom_col(
            mapping = aes(
                fill = n
            )
        ) + 
        ##  Adjust sizing  ##
        coord_fixed(
            ratio = 15
        ) +
        ##  Adjust coloring  ##
        scale_fill_gradient(
            low = "lightgray",
            high = "steelblue"
        )
    
}
# lenderedge %>% getAccountTypePlotFunnel(quo(funded), quo(qualified))

In [178]:
getAccountLengthPlotFunnel = function (df, numerator.quo, denominator.quo) {

    ##  With full df, sum Qualified Volume by State.  ##
    summary.df = df %>%
        filter(
            qualified == 1
        ) %>% 
        group_by(
            raw_accountlength
        ) %>% 
        summarize(
            n = n(),
            metric = sum(!!numerator.quo)/sum(!!denominator.quo)
        ) %>% 
        ungroup() %>% 
        filter(
            n >= 1 &
            metric < 1
        )
    
    ##  Bar Plot  ##
    summary.df %>% 
        ggplot(
            mapping = aes(
                x = raw_accountlength,
                y = metric
            )
        ) +
        geom_col(
            mapping = aes(
                fill = n
            )
        ) + 
        ##  Adjust sizing  ##
        coord_fixed(
            ratio = 5
        ) +
        ##  Adjust coloring  ##
        scale_fill_gradient(
            low = "lightgray",
            high = "steelblue"
        )
    
}
# lenderedge %>% getAccountLengthPlotFunnel(quo(funded), quo(qualified))

### DOB

- FR for older customers drops off.

In [167]:
# getDOBPlotFunnel = function (df, numerator.quo, denominator.quo) {

#     ##  With full df, sum Qualified Volume by State.  ##
#     summary.df = df %>%
#         filter(
#             qualified == 1
#         ) %>% 
#         group_by(
#             dob = ((.$lead_time %>% as.Date() - .$raw_dateofbirth %>% as.Date()) %>% as.numeric())/365,
#             dob = dob %>% floor()
#         ) %>% 
#         summarize(
#             n = n(),
#             metric = sum(!!numerator.quo)/sum(!!denominator.quo)
#         ) %>% 
#         ungroup() %>% 
#         filter(
#             n >= 1 &
#             metric < 1
#         )
    
#     ##  Bar Plot  ##
#     summary.df %>% 
#         ggplot(
#             mapping = aes(
#                 x = dob,
#                 y = metric
#             )
#         ) +
#         geom_col(
#             mapping = aes(
#                 fill = n
#             )
#         ) + 
#         ##  Adjust sizing  ##
#         coord_fixed(
#             ratio = 100
#         ) +
#         ##  Adjust coloring  ##
#         scale_fill_gradient(
#             low = "lightgray",
#             high = "steelblue"
#         )
    
# }

# lenderedge %>% getDOBPlotFunnel(quo(funded), quo(qualified))

# Bivariate
### Do any variable pairs have a direct correlation with funding?

In [ ]:
# https://learnr.wordpress.com/2010/01/26/ggplot2-quick-heatmap-plotting/

#### GMI <> DOB <> ZIP

# Decision Tree

In [ ]:
# https://www.statmethods.net/advstats/cart.html
library(rpart)
library(rattle)
library(rpart.plot)

In [ ]:
lenderedge.train = lenderedge %>% filter(appldate < '2019-10-01' %>% as.Date())
lenderedge.test = lenderedge %>% filter(appldate >= '2019-10-01' %>% as.Date())

In [ ]:
lenderedge.test.1 = lenderedge %>%
    filter(
        appldate >= '2019-10-01' %>% as.Date() &
        appldate <  '2019-10-08' %>% as.Date()
    )

lenderedge.test.2 = lenderedge %>%
    filter(
        appldate >= '2019-10-14' %>% as.Date() &
        appldate < '2019-10-31' %>% as.Date()
    )

lenderedge.test.2.1 = lenderedge %>%
    filter(
        appldate >= '2019-10-14' %>% as.Date() &
        appldate < '2019-10-22' %>% as.Date()
    )

lenderedge.test.2.2 = lenderedge %>%
    filter(
        appldate >= '2019-10-22' %>% as.Date() &
        appldate < '2019-10-31' %>% as.Date()
    )

### Train Tree: Aug & Sept

In [ ]:
# ## Population

# rpart(
#     formula = funded ~
#         click_id +
#         raw_abaroutingnumber.is.valid +
#         raw_accountnumber.is.valid +
#         raw_grossmonthlyincome +
#         raw_statecode,
# #         age,
#     data = lenderedge.train %>%
#         mutate(
#             funded = ifelse(funded, "Funded", "Denied")
#         ),
#     method = 'class',    ##  classification tree
#     control = rpart.control(
#         minsplit = 2,
#         minbucket = 10,
#         cp = 0.0002
#     )
# ) %T>% 
# #     summary() %T>%
# #     plot() %T>%
# #     text() %T>%
#     fancyRpartPlot()

In [ ]:
# ## Population.Node #14

# rpart(
#     formula = funded ~
#         click_id +
#         raw_abaroutingnumber.is.valid +
#         raw_accountnumber.is.valid +
#         raw_grossmonthlyincome +
#         raw_statecode,
#     data = lenderedge.train %>%
#         mutate(
#             funded = ifelse(funded, "Funded", "Denied")
#         ) %>% 
#         filter(
#             raw_grossmonthlyincome >= 2162.5 &
#             raw_statecode %in% c('AL','DC','DE','HI','IN','KY','LA','ME','MI','MO','MS','MT','OK','OR','RI','TN','TX','UT','VA','WI','WY') &
#             (
#                 click_id %in% c('1716','1726','1727','1728','1730','1731','1732','1733','1735','1736','1737','1743','1747','1749','1757','1758') |
#                 click_id == '%201749' |
#                 click_id == 'None'
#             )
#         ),
#     method = 'class',    ##  classification tree
#     control = rpart.control(
#         minsplit = 2,
#         minbucket = 5,
#         cp = 0.0004
#     )
# ) %T>% 
# #     summary() %T>%
# #     plot() %T>%
# #     text() %T>%
#     fancyRpartPlot()

In [ ]:
# ## Population.Node #14.Node #2

# rpart(
#     formula = funded ~
#         click_id +
#         raw_abaroutingnumber.is.valid +
#         raw_accountnumber.is.valid +
#         raw_grossmonthlyincome +
#         raw_statecode,
#     data = lenderedge.train %>%
#         mutate(
#             funded = ifelse(funded, "Funded", "Denied")
#         ) %>% 
#         filter(
#             raw_grossmonthlyincome >= 2162.5 &
#             raw_statecode %in% c('AL','DC','DE','HI','IN','KY','LA','ME','MI','MO','MS','MT','OK','OR','RI','TN','TX','UT','VA','WI','WY') &
#             (
#                 click_id %in% c('1716','1726','1727','1728','1730','1731','1732','1733','1735','1736','1737','1743','1747','1749','1757','1758') |
#                 click_id == '%201749' |
#                 click_id == 'None'
#             ) &
#             raw_grossmonthlyincome <  4002
#         ),
#     method = 'class',    ##  classification tree
#     control = rpart.control(
#         minsplit = 2,
#         minbucket = 3,
#         cp = 0.0007
#     )
# ) %T>% 
# #     summary() %T>%
# #     plot() %T>%
# #     text() %T>%
#     fancyRpartPlot()

In [ ]:
# ## Population.Node #14.Node #2.Node #28

# rpart(
#     formula = funded ~
#         click_id +
#         raw_abaroutingnumber.is.valid +
#         raw_accountnumber.is.valid +
#         raw_grossmonthlyincome +
#         raw_statecode,
#     data = lenderedge.train %>%
#         mutate(
#             funded = ifelse(funded, "Funded", "Denied")
#         ) %>% 
#         filter(
#             raw_grossmonthlyincome >= 2162.5 &
#             raw_statecode %in% c('AL','DC','DE','HI','IN','KY','LA','ME','MI','MO','MS','MT','OK','OR','RI','TN','TX','UT','VA','WI','WY') &
#             (
#                 click_id %in% c('1716','1726','1727','1728','1730','1731','1732','1733','1735','1736','1737','1743','1747','1749','1757','1758') |
#                 click_id == '%201749' |
#                 click_id == 'None'
#             ) &
#             raw_grossmonthlyincome <  4002 &
#             raw_statecode %in% c('AL','IN','KY','ME','MI','MS','OR','RI','TX','UT','WI') &
#             (
#                 click_id %in% c('1716','1727','1732','1735','1736','1743','1747','1758') |
#                 click_id == '%201749' |
#                 click_id == 'None'
#             ) &
#             raw_grossmonthlyincome >= 2215.5
#         ),
#     method = 'class',    ##  classification tree
#     control = rpart.control(
#         minsplit = 2,
#         minbucket = 2,
#         cp = 0.0009
#     )
# ) %T>% 
# #     summary() %T>%
# #     plot() %T>%
# #     text() %T>%
#     fancyRpartPlot()

In [ ]:
# ## Population.Node #6

# rpart(
#     formula = funded ~
#         click_id +
#         raw_abaroutingnumber.is.valid +
#         raw_accountnumber.is.valid +
#         raw_grossmonthlyincome +
#         raw_statecode,
#     data = lenderedge.train %>%
#         mutate(
#             funded = ifelse(funded, "Funded", "Denied")
#         ) %>% 
#         filter(
#             raw_grossmonthlyincome >= 2162.5 &
#             raw_statecode %in% c('AL','DC','DE','HI','IN','KY','LA','ME','MI','MO','MS','MT','OK','OR','RI','TN','TX','UT','VA','WI','WY')
#         ),
#     method = 'class',    ##  classification tree
#     control = rpart.control(
#         minsplit = 2,
#         minbucket = 5,
#         cp = 0.0004
#     )
# ) %T>% 
# #     summary() %T>%
# #     plot() %T>%
# #     text() %T>%
#     fancyRpartPlot()

In [ ]:
# ## Population.Node #6.Node #10

# rpart(
#     formula = funded ~
#         click_id +
#         raw_abaroutingnumber.is.valid +
#         raw_accountnumber.is.valid +
#         raw_grossmonthlyincome +
#         raw_statecode,
#     data = lenderedge.train %>%
#         mutate(
#             funded = ifelse(funded, "Funded", "Denied")
#         ) %>% 
#         filter(
#             raw_grossmonthlyincome >= 2162.5 &
#             raw_statecode %in% c('AL','DC','DE','HI','IN','KY','LA','ME','MI','MO','MS','MT','OK','OR','RI','TN','TX','UT','VA','WI','WY') &
#             raw_grossmonthlyincome < 3946 &
#             raw_statecode %in% c('AL','IN','KY','ME','MI','MS','OR','TX','UT') &
#             (
#                 click_id %in% c('1716','1718','1726','1728','1730','1731','1732','1733','1734','1735','1736','1737','1741','1742','1743','1744','1746','1747','1749','1757','1758') |
#                 click_id == '%201749' |
#                 click_id == 'None'
#             )
#         ),
#     method = 'class',    ##  classification tree
#     control = rpart.control(
#         minsplit = 2,
#         minbucket = 3,
#         cp = 0.0001
#     )
# ) %T>% 
# #     summary() %T>%
# #     plot() %T>%
# #     text() %T>%
#     fancyRpartPlot()

In [ ]:
# ## Population.Node #2

# rpart(
#     formula = funded ~
#         click_id +
#         raw_abaroutingnumber.is.valid +
#         raw_accountnumber.is.valid +
#         raw_grossmonthlyincome +
#         raw_statecode,
#     data = lenderedge.train %>%
#         mutate(
#             funded = ifelse(funded, "Funded", "Denied")
#         ) %>% 
#         filter(
#             raw_grossmonthlyincome < 2163
#         ),
#     method = 'class',    ##  classification tree
#     control = rpart.control(
#         minsplit = 2,
#         minbucket = 5,
#         cp = 0.0004
#     )
# ) %T>% 
# #     summary() %T>%
# #     plot() %T>%
# #     text() %T>%
#     fancyRpartPlot()

In [ ]:
# ## Population.Node #2.Node #6

# rpart(
#     formula = funded ~
#         click_id +
#         raw_abaroutingnumber.is.valid +
#         raw_accountnumber.is.valid +
#         raw_grossmonthlyincome +
#         raw_statecode,
#     data = lenderedge.train %>%
#         mutate(
#             funded = ifelse(funded, "Funded", "Denied")
#         ) %>% 
#         filter(
#             raw_grossmonthlyincome < 2163 &
#             raw_statecode %in% c('AZ','FL','IL','IN','MI','MN','NM','NV','OH','OR','TN','WA') &
#             click_id %in% c('1716','1718','1727','1728','1730','1731','1732','1734','1735','1736','1737','1741','1742','1743','1744','1745','1746','1748','1749','1757','1758') &
#             click_id != '%201749' &
#             click_id != 'None'
#         ),
#     method = 'class',    ##  classification tree
#     control = rpart.control(
#         minsplit = 2,
#         minbucket = 5,
#         cp = 0.0004
#     )
# ) %T>% 
# #     summary() %T>%
# #     plot() %T>%
# #     text() %T>%
#     fancyRpartPlot()

In [ ]:
# ## Population.Node #2.Node #6.Node #4

# rpart(
#     formula = funded ~
#         click_id +
#         raw_abaroutingnumber.is.valid +
#         raw_accountnumber.is.valid +
#         raw_grossmonthlyincome +
#         raw_statecode,
#     data = lenderedge.train %>%
#         mutate(
#             funded = ifelse(funded, "Funded", "Denied")
#         ) %>% 
#         filter(
#             raw_grossmonthlyincome < 2163 &
#             raw_statecode %in% c('AZ','FL','IL','IN','MI','MN','NM','NV','OH','OR','TN','WA') &
#             click_id %in% c('1716','1718','1727','1728','1730','1731','1732','1734','1735','1736','1737','1741','1742','1743','1744','1745','1746','1748','1749','1757','1758') &
#             click_id != '%201749' &
#             click_id != 'None' &
#             click_id %in% c('1728','1730','1731','1732','1734','1736','1741','1744','1745','1746','1749','1758') &
#             raw_statecode %in% c('AZ','FL','IN','MI','NV','TN','WA')
#         ),
#     method = 'class',    ##  classification tree
#     control = rpart.control(
#         minsplit = 2,
#         minbucket = 1,
#         cp = 0.0004
#     )
# ) %T>% 
# #     summary() %T>%
# #     plot() %T>%
# #     text() %T>%
#     fancyRpartPlot()

### Test Tree

### Train Tree: FR ~ Early Oct!!!!!

In [ ]:
lenderedge.test.1 %>% filter(qualified == 1) %>% summarize(mean(funded))
lenderedge.test.2 %>% filter(qualified == 1) %>% summarize(mean(funded))

In [ ]:
##  Train vs Test
lenderedge.test.1 %>% 
    filter(
        qualified == 1
    ) %>% 
    group_by(
        low.conversion.group = 
            (
                raw_statecode %in% c('DE','ID','MO','TN','UT','WY') |
                (
                    raw_statecode %in% c('KY','MS','OR','TX','VA','WI') &
                    raw_grossmonthlyincome < 1775
                ) |
                (
                    raw_statecode %in% c('IN','KY','MI','MN','OH','WA') &
                    raw_grossmonthlyincome <= 1900
                ) |
                (
                    raw_statecode %in% c('FL','ND','TX','UT') &
                    raw_grossmonthlyincome >= 9625
                ) |
                (
                    raw_statecode %in% c('IL','NM','TX') &
                    raw_grossmonthlyincome <= 3825 &
                    raw_grossmonthlyincome >= 3455                    
#                 ) |
#                 (
#                     raw_statecode %in% c('FL','IL','NM','TX') &
#                     raw_grossmonthlyincome <= 3455 &
#                     raw_grossmonthlyincome >= 3275           
#                 ) |
#                 (
#                     raw_statecode %in% c('AL','ID','ND','NE','NV','UT') &
#                     raw_grossmonthlyincome >= 6000
                )
            )
    ) %>% 
    summarize(
        qualified = n(),
#         bv = sum(bankverified),
#         bv.q = mean(bankverified),
        funded.loans = sum(funded),
        funding.rate = mean(funded)
    )

lenderedge.test.2 %>% 
    filter(
        qualified == 1
    ) %>% 
    group_by(
        low.conversion.group = 
            (
                raw_statecode %in% c('DE','ID','MO','TN','UT','WY') |
                (
                    raw_statecode %in% c('KY','MS','OR','TX','VA','WI') &
                    raw_grossmonthlyincome < 1775
                ) |
                (
                    raw_statecode %in% c('IN','KY','MI','MN','OH','WA') &
                    raw_grossmonthlyincome <= 1900
                ) |
                (
                    raw_statecode %in% c('FL','ND','TX','UT') &
                    raw_grossmonthlyincome >= 9625
                ) |
                (
                    raw_statecode %in% c('IL','NM','TX') &
                    raw_grossmonthlyincome <= 3825 &
                    raw_grossmonthlyincome >= 3455                    
#                 ) |
#                 (
#                     raw_statecode %in% c('FL','IL','NM','TX') &
#                     raw_grossmonthlyincome <= 3455 &
#                     raw_grossmonthlyincome >= 3275           
#                 ) |
#                 (
#                     raw_statecode %in% c('AL','ID','ND','NE','NV','UT') &
#                     raw_grossmonthlyincome >= 6000
                )
            )
    ) %>% 
    summarize(
        qualified = n(),
#         bv = sum(bankverified),
#         bv.q = mean(bankverified),
        funded.loans = sum(funded),
        funding.rate = mean(funded)
    )

In [ ]:
# ## Population

# rpart(
#     formula = funded ~
#         raw_grossmonthlyincome +
#         raw_statecode,
#     data = lenderedge.test.1 %>% 
#         filter(
#             qualified == 1
#         ) %>% 
#         mutate(
#             funded = ifelse(funded, "Funded", "Denied")
#         ),
#     method = 'class',    ##  classification tree
#     control = rpart.control(
#         minsplit = 2,
#         minbucket = 12,
#         cp = 0.0001
#     )
# ) %T>%
#     fancyRpartPlot(
#     )

In [ ]:
# ## Population.Node #2

# rpart(
#     formula = funded ~
#         raw_grossmonthlyincome +
#         raw_statecode,
#     data = lenderedge.test.1 %>% 
#         filter(
#             qualified == 1 &
#             raw_statecode %in% c('DE','ID','KY','MO','MS','OR','TN','TX','UT','VA','WI','WY')
#         ) %>% 
#         mutate(
#             funded = ifelse(funded, "Funded", "Denied")
#         ),
#     method = 'class',    ##  classification tree
#     control = rpart.control(
#         minsplit = 2,
#         minbucket = 2,
#         cp = 0.0001
#     )
# ) %T>%
#     fancyRpartPlot(
#     )

In [ ]:
# ## Population.Node #6

# rpart(
#     formula = funded ~
#         raw_grossmonthlyincome +
#         raw_statecode,
#     data = lenderedge.test.1 %>% 
#         filter(
#             qualified == 1 &
#             raw_statecode %in% c('AL','AZ','DC','FL','GA','IL','IN','MI','MN','ND','NE','NM','NV','OH','SD','WA') &
#             raw_grossmonthlyincome < 5925
#         ) %>% 
#         mutate(
#             funded = ifelse(funded, "Funded", "Denied")
#         ),
#     method = 'class',    ##  classification tree
#     control = rpart.control(
#         minsplit = 2,
#         minbucket = 11,
#         cp = 0.0001
#     )
# ) %T>%
#     fancyRpartPlot(
#     )

In [ ]:
# ## Population.Node #14

# rpart(
#     formula = funded ~
#         raw_grossmonthlyincome +
#         raw_statecode,
#     data = lenderedge.test.1 %>% 
#         filter(
#             qualified == 1 &
#             raw_grossmonthlyincome >= 2684 &
#             raw_statecode %in% c('AL','FL','IN','MI','NM','OH','SD','WA')
#         ) %>% 
#         mutate(
#             funded = ifelse(funded, "Funded", "Denied")
#         ),
#     method = 'class',    ##  classification tree
#     control = rpart.control(
#         minsplit = 2,
#         minbucket = 11,
#         cp = 0.0001
#     )
# ) %T>%
#     fancyRpartPlot(
#     )

### Train Tree: FR ~ Late Oct

In [ ]:
# ## Population

# rpart(
#     formula = funded ~
#         raw_grossmonthlyincome +
#         raw_statecode,
#     data = lenderedge.test.2.2 %>% 
#         filter(
#             qualified == 1
#         ) %>% 
#         mutate(
#             funded = ifelse(funded, "Funded", "Denied")
#         ),
#     method = 'class',    ##  classification tree
#     control = rpart.control(
#         minsplit = 2,
#         minbucket = 10,
#         cp = 0.0001
#     )
# ) %T>%
#     fancyRpartPlot(
#     )

In [ ]:
# ## Population

# rpart(
#     formula = funded ~
#         raw_grossmonthlyincome +
#         raw_statecode,
#     data = lenderedge.test.2.2 %>% 
#         filter(
#             qualified == 1 &
#             (
#                 raw_statecode %in% c('AL','FL','ID','IL','ND','NE','NM','NV','TX','UT') &
#                 raw_grossmonthlyincome < 4316.5
#             )
#         ) %>% 
#         mutate(
#             funded = ifelse(funded, "Funded", "Denied")
#         ),
#     method = 'class',    ##  classification tree
#     control = rpart.control(
#         minsplit = 2,
#         minbucket = 1,
#         cp = 0.00001
#     )
# ) %T>%
#     fancyRpartPlot()

#### Node Level

In [ ]:
##  Exclusion Logic
##  Exclude 2.6.4

raw_grossmonthlyincome < 2163 &
raw_statecode %in% c('AZ','FL','IN','MI','NV','TN','WA') &
click_id %in% c('1728','1730','1731','1732','1734','1736','1741','1744','1745','1746','1749','1758') &
click_id != '%201749' &
click_id != 'None'


##  Exclude 2.6.10

raw_grossmonthlyincome < 2163 &
raw_statecode %in% c('IL','MN','NM','OH','OR') &
click_id %in% c('1731','1732','1734','1741','1745','1746','1758') &
click_id != '%201749' &
click_id != 'None'

##  Exclude 2.2

raw_grossmonthlyincome < 2163 &
raw_statecode %in% c('DC','DE','HI','KS','KY','LA','ME','MO','MS','MT','ND','NE','OK','RI','SD','TX','UT','VA','WI','WY')

##  Exclude 6.10.2

raw_grossmonthlyincome >= 2162.5 &
raw_grossmonthlyincome < 3946 &
raw_statecode %in% c('AL','IN','KY','ME','MI','MS','OR','TX','UT') &
click_id %in% c('1726','1728','1730','1731','1733','1737','1741','1742','1744','1746','1747','1749','1757')

##  Exclude 6.4

raw_grossmonthlyincome >= 2162.5 &
raw_grossmonthlyincome < 4001.5 &
raw_statecode %in% c('AL','DC','DE','HI','IN','KY','LA','ME','MI','MO','MS','MT','OK','OR','RI','TN','TX','UT','VA','WI','WY')


##  Exclude 14.2.2

raw_grossmonthlyincome >= 2162.5 &
raw_grossmonthlyincome < 4002 &
raw_statecode %in% c('DC','DE','HI','LA','MO','MT','OK','TN','VA','WY') &
(
    click_id %in% c('1716','1726','1727','1728','1730','1731','1732','1733','1735','1736','1737','1743','1747','1749','1757','1758') |
    click_id == '%201749' |
    click_id == 'None'
)

##  Exclude 14.2.6

raw_grossmonthlyincome >= 2162.5 &
raw_grossmonthlyincome < 4002 &
raw_statecode %in% c('AL','IN','KY','ME','MI','MS','OR','RI','TX','UT','WI') &
click_id %in% c('1726','1728','1730','1731','1733','1737','1749','1757')

In [ ]:
##  Exclusion Logic Details on Test Set
lenderedge.test %>% 
    mutate(
        test.exclude.2.6.4 = 
        (
            ##  Exclude 2.6.4

            raw_grossmonthlyincome < 2163 &
            raw_statecode %in% c('AZ','FL','IN','MI','NV','TN','WA') &
            click_id %in% c('1728','1730','1731','1732','1734','1736','1741','1744','1745','1746','1749','1758') &
            click_id != '%201749' &
            click_id != 'None'
        ),
        test.exclude.2.6.10 = 
        (
            ##  Exclude 2.6.10

            raw_grossmonthlyincome < 2163 &
            raw_statecode %in% c('IL','MN','NM','OH','OR') &
            click_id %in% c('1731','1732','1734','1741','1745','1746','1758') &
            click_id != '%201749' &
            click_id != 'None'
        ),
        test.exclude.2.2 = 
        (
            ##  Exclude 2.2

            raw_grossmonthlyincome < 2163 &
            raw_statecode %in% c('DC','DE','HI','KS','KY','LA','ME','MO','MS','MT','ND','NE','OK','RI','SD','TX','UT','VA','WI','WY')
        ),
        test.exclude.6.10.2 = 
        (
            ##  Exclude 6.10.2

            raw_grossmonthlyincome >= 2162.5 &
            raw_grossmonthlyincome < 3946 &
            raw_statecode %in% c('AL','IN','KY','ME','MI','MS','OR','TX','UT') &
            click_id %in% c('1726','1728','1730','1731','1733','1737','1741','1742','1744','1746','1747','1749','1757')
        ),
        test.exclude.6.4 = 
        (
            ##  Exclude 6.4

            raw_grossmonthlyincome >= 2162.5 &
            raw_grossmonthlyincome < 4001.5 &
            raw_statecode %in% c('AL','DC','DE','HI','IN','KY','LA','ME','MI','MO','MS','MT','OK','OR','RI','TN','TX','UT','VA','WI','WY')
        ),
        test.exclude.14.2.2 = 
        (
            ##  Exclude 14.2.2

            raw_grossmonthlyincome >= 2162.5 &
            raw_grossmonthlyincome < 4002 &
            raw_statecode %in% c('DC','DE','HI','LA','MO','MT','OK','TN','VA','WY') &
            (
                click_id %in% c('1716','1726','1727','1728','1730','1731','1732','1733','1735','1736','1737','1743','1747','1749','1757','1758') |
                click_id == '%201749' |
                click_id == 'None'
            )
        ),
        test.exclude.14.2.6 = 
        (
            ##  Exclude 14.2.6

            raw_grossmonthlyincome >= 2162.5 &
            raw_grossmonthlyincome < 4002 &
            raw_statecode %in% c('AL','IN','KY','ME','MI','MS','OR','RI','TX','UT','WI') &
            click_id %in% c('1726','1728','1730','1731','1733','1737','1749','1757')
        )
    ) %>% 
    filter(
        qualified == 1
    ) %>% 
    summarize(
        vol.2.6.4 = sum(test.exclude.2.6.4),
        vol.2.6.10 = sum(test.exclude.2.6.10),
        vol.2.2 = sum(test.exclude.2.2),
        vol.6.10.2 = sum(test.exclude.6.10.2),
        vol.6.4 = sum(test.exclude.6.4),
        vol.14.2.2 = sum(test.exclude.14.2.2),
        vol.14.2.6 = sum(test.exclude.14.2.6),
        fund.2.6.4 = sum(test.exclude.2.6.4 & funded),
        fund.2.6.10 = sum(test.exclude.2.6.10 & funded),
        fund.2.2 = sum(test.exclude.2.2 & funded),
        fund.6.10.2 = sum(test.exclude.6.10.2 & funded),
        fund.6.4 = sum(test.exclude.6.4 & funded),
        fund.14.2.2 = sum(test.exclude.14.2.2 & funded),
        fund.14.2.6 = sum(test.exclude.14.2.6 & funded),
        volume = n(),
        bv.q = mean(bankverified),
        fr = mean(funded)
    ) %>% 
    ungroup() %>% 
    summarize(
        fr.2.6.4 = fund.2.6.4/vol.2.6.4,
        fr.2.6.10 = fund.2.6.10/vol.2.6.10,
        fr.2.2 = fund.2.2/vol.2.2,
        fr.6.10.2 = fund.6.10.2/vol.6.10.2,
        fr.6.4 = fund.6.4/vol.6.4,
        fr.14.2.2 = fund.14.2.2/vol.14.2.2,
        fr.14.2.6 = fund.14.2.6/vol.14.2.6,
        fr
    )

In [ ]:
##  Exclusion Logic Summary on Test Set
lenderedge.test %>% 
    mutate(
        test.exclude = 
#         (
#             ##  Exclude 2.6.4

#             raw_grossmonthlyincome < 2163 &
#             raw_statecode %in% c('AZ','FL','IN','MI','NV','TN','WA') &
#             click_id %in% c('1728','1730','1731','1732','1734','1736','1741','1744','1745','1746','1749','1758') &
#             click_id != '%201749' &
#             click_id != 'None'
#         ) |
#         (
#             ##  Exclude 2.6.10

#             raw_grossmonthlyincome < 2163 &
#             raw_statecode %in% c('IL','MN','NM','OH','OR') &
#             click_id %in% c('1731','1732','1734','1741','1745','1746','1758') &
#             click_id != '%201749' &
#             click_id != 'None'
#         ) |
        (
            ##  Exclude 2.2

            raw_grossmonthlyincome < 2163 &
            raw_statecode %in% c('DC','DE','HI','KS','KY','LA','ME','MO','MS','MT','ND','NE','OK','RI','SD','TX','UT','VA','WI','WY')
        ) |
        (
            ##  Exclude 6.10.2

            raw_grossmonthlyincome >= 2162.5 &
            raw_grossmonthlyincome < 3946 &
            raw_statecode %in% c('AL','IN','KY','ME','MI','MS','OR','TX','UT') &
            click_id %in% c('1726','1728','1730','1731','1733','1737','1741','1742','1744','1746','1747','1749','1757')
        ) |
#         (
#             ##  Exclude 6.4

#             raw_grossmonthlyincome >= 2162.5 &
#             raw_grossmonthlyincome < 4001.5 &
#             raw_statecode %in% c('AL','DC','DE','HI','IN','KY','LA','ME','MI','MO','MS','MT','OK','OR','RI','TN','TX','UT','VA','WI','WY')
#         ) |
#         (
#             ##  Exclude 14.2.2

#             raw_grossmonthlyincome >= 2162.5 &
#             raw_grossmonthlyincome < 4002 &
#             raw_statecode %in% c('DC','DE','HI','LA','MO','MT','OK','TN','VA','WY') &
#             (
#                 click_id %in% c('1716','1726','1727','1728','1730','1731','1732','1733','1735','1736','1737','1743','1747','1749','1757','1758') |
#                 click_id == '%201749' |
#                 click_id == 'None'
#             )
#         ) |
        (
            ##  Exclude 14.2.6

            raw_grossmonthlyincome >= 2162.5 &
            raw_grossmonthlyincome < 4002 &
            raw_statecode %in% c('AL','IN','KY','ME','MI','MS','OR','RI','TX','UT','WI') &
            click_id %in% c('1726','1728','1730','1731','1733','1737','1749','1757')
        )
    ) %>% 
    filter(
        qualified == 1
    ) %>% 
    group_by(
        test.exclude
    ) %>% 
    summarize(
        volume = n(),
        bv.q = mean(bankverified),
        fr = mean(funded)
    )

In [ ]:
##  Exclusion Logic Details on Test Set
lenderedge.test %>%         
    filter(
        qualified == 1
    ) %>% 
    summarize(
        volume = n(),
        bv.q.reported = mean(bankverified),
        fr.reported = mean(funded)
    )

lenderedge.test %>% 
    mutate(
        test.exclude = 
#         (
#             ##  Exclude 2.6.4

#             raw_grossmonthlyincome < 2163 &
#             raw_statecode %in% c('AZ','FL','IN','MI','NV','TN','WA') &
#             click_id %in% c('1728','1730','1731','1732','1734','1736','1741','1744','1745','1746','1749','1758') &
#             click_id != '%201749' &
#             click_id != 'None'
#         ) |
#         (
#             ##  Exclude 2.6.10

#             raw_grossmonthlyincome < 2163 &
#             raw_statecode %in% c('IL','MN','NM','OH','OR') &
#             click_id %in% c('1731','1732','1734','1741','1745','1746','1758') &
#             click_id != '%201749' &
#             click_id != 'None'
#         ) |
        (
            ##  Exclude 2.2

            raw_grossmonthlyincome < 2163 &
            raw_statecode %in% c('DC','DE','HI','KS','KY','LA','ME','MO','MS','MT','ND','NE','OK','RI','SD','TX','UT','VA','WI','WY')
        ) |
        (
            ##  Exclude 6.10.2

            raw_grossmonthlyincome >= 2162.5 &
            raw_grossmonthlyincome < 3946 &
            raw_statecode %in% c('AL','IN','KY','ME','MI','MS','OR','TX','UT') &
            click_id %in% c('1726','1728','1730','1731','1733','1737','1741','1742','1744','1746','1747','1749','1757')
        ) |
#         (
#             ##  Exclude 6.4

#             raw_grossmonthlyincome >= 2162.5 &
#             raw_grossmonthlyincome < 4001.5 &
#             raw_statecode %in% c('AL','DC','DE','HI','IN','KY','LA','ME','MI','MO','MS','MT','OK','OR','RI','TN','TX','UT','VA','WI','WY')
#         ) |
#         (
#             ##  Exclude 14.2.2

#             raw_grossmonthlyincome >= 2162.5 &
#             raw_grossmonthlyincome < 4002 &
#             raw_statecode %in% c('DC','DE','HI','LA','MO','MT','OK','TN','VA','WY') &
#             (
#                 click_id %in% c('1716','1726','1727','1728','1730','1731','1732','1733','1735','1736','1737','1743','1747','1749','1757','1758') |
#                 click_id == '%201749' |
#                 click_id == 'None'
#             )
#         ) |
        (
            ##  Exclude 14.2.6

            raw_grossmonthlyincome >= 2162.5 &
            raw_grossmonthlyincome < 4002 &
            raw_statecode %in% c('AL','IN','KY','ME','MI','MS','OR','RI','TX','UT','WI') &
            click_id %in% c('1726','1728','1730','1731','1733','1737','1749','1757')
        )
    ) %>% 
    filter(
        qualified == 1 &
        !test.exclude
    ) %>% 
    summarize(
        volume = n(),
        bv.q.proposed = mean(bankverified),
        fr.proposed = mean(funded)
    )

# Random Forest

In [ ]:
library(randomForest)

In [ ]:
lenderedge.train.rf = lenderedge.train %>%
    filter(
        !is.na(raw_statecode)
    ) %>%
    mutate(
        funded = ifelse(funded, "Funded", "Denied")
    ) %>% 
    mutate_if(
        .predicate = is.character,
        .funs = as.factor  ##  requires factors
    )

In [ ]:
lenderedge.rf = randomForest(
    formula = funded ~
        click_id +
#         raw_abaroutingnumber.is.valid +
#         raw_accountnumber.is.valid +
        raw_grossmonthlyincome +
        raw_statecode,
    data = lenderedge.train.rf,
    mtry = 2,
    ntree = 250,
    sampsize = ceiling(0.632*nrow(lenderedge.train)),
    replace = TRUE,
    importance = TRUE
)
lenderedge.rf

In [ ]:
lenderedge.rf.wt = randomForest(
    formula = funded ~
        click_id +
        raw_abaroutingnumber.is.valid +
        raw_accountnumber.is.valid +
        raw_grossmonthlyincome +
        raw_statecode,
    data = lenderedge.train.rf,
    mtry = 2,
    ntree = 500,
    strata = lenderedge.train.rf$funded,
#     sampsize = ceiling(0.8*nrow(lenderedge.train)),
    sampsize = c(
        200,  ##  Denied
        100   ##  Approved
    ),
    replace = TRUE,
    importance = TRUE
)
lenderedge.rf.wt

In [ ]:
approved.n = seq(
    from = 100,
    to = 1000,
    by = 100
)

oob.error = vector(length = length(approved.n))

container = data.frame(
    approved.n,
    oob.error
)

for (i in 1:nrow(container)) {
    
    lenderedge.rf.wt = randomForest(
        formula = funded ~
            click_id +
            raw_abaroutingnumber.is.valid +
            raw_accountnumber.is.valid +
            raw_grossmonthlyincome +
            raw_statecode,
        data = lenderedge.train.rf,
        mtry = 2,
        ntree = 500,
        strata = lenderedge.train.rf$funded,
    #     sampsize = ceiling(0.8*nrow(lenderedge.train)),
        sampsize = c(
            200,                      ##  Denied
            container$approved.n[1]   ##  Approved
        ),
        replace = TRUE,
        importance = TRUE
    )
    
    container$oob.error[i] = lenderedge.rf.wt$err.rate %>% as.data.frame() %>% summarize(mu = mean(Denied)*mean(Funded)) %>% .$mu
}